## Концепт сплитовалки трафика

In [1]:
import numpy as np

#### У нас есть какие-то идентификаторы, по которым мы собираемся делать сплит.

In [380]:
device_id = [np.random.randint(1,999999999) for i in range(1000000)]

#### Также у нас есть конфиги экспериментов - (название, соль, доля от общего трафика, объем эксперимента), и каскад фичей - фичи, которые используются по дефолту или в рамках групп в экспериментах.

In [392]:
cash_ids = {1:{'base_group':1, 'experiments':{}}}


config = {'experiments':
              {1:{'name': 'Эксперимент 1',
                  'description': 'Провести эксперимент разбиения трафика на 3 группы, вероятность попадания участника в данных эксперимент 50%, общий объем эксперимента 1000',
                  'salt': 42,
                  'p': 0.1,
                  'size': 30000,
                  'actual_size':0,
                  'status':'active',
                  'groups': [{'group1':{'description':'Группа 1 - свойства = Синяя кнопка, 3 меню, размер страницы 100%'}},
                             {'group2':{'description':'Группа 2 - свойства = Красная кнопка, 2 меню, размер страницы 90%'}},
                             {'group3':{'description':'Группа 3 - свойства = Зеленая кнопка, 1 меню, размер страницы 120%'}},
                            ],
                 'cross_relates':None,
                 'params':{'base_group':[13,22,45,47,52,54,57,60,73,99]}}
              }}

features_cascade = {'default':{'backgroung_color':'white', 'buttom_color':'red', 'text_color':'black','size':100, 'price':50, 'delta':10},
                    1:[{'group1':{'backgroung_color':'yellow','text_color':'red'}},
                       {'group2':{'backgroung_color':'light-blue','text_color':'white'}},
                       {'group3':{'backgroung_color':'light-breen','text_color':'brown'}}]
                   }

In [393]:
class super_service:
    def __init__(self, config = {}, cash_ids = {}, features_cascade = {}):
        self.config = config
        self.cash_ids = cash_ids
        self.features_cascade = features_cascade
        
    def calculate_group(self, id_):
        data = {id_:{'base_group':id_%100,'experiments':{}}}
                
        if id_%100 < 10:
            self.save_cash(data)
            return data
        else:
            data[id_]['experiments'] = self.include_in_experiment(id_)
            self.save_cash(data)
            return data
            
    def include_in_experiment(self, id_):
        base_group = id_%100
        tmp = {}
        for k,v in self.config['experiments'].items():
            if base_group in v['params']['base_group'] and v['actual_size']<v['size']:
                group_exp = (id_+v['salt'])%len(v['groups'])
                tmp[k] = {list(v['groups'][group_exp])[0]}
                self.config['experiments'][k]['actual_size'] +=1
            else:
                tmp[k] = {}
        return tmp
            
    def save_cash(self, data):
        '''data = dict('id_':'group')'''
        self.cash_ids.update(data)
        
    def get_group(self, id_):
        if id_ in self.cash_ids.keys():
            return {id_:self.cash_ids[id_]}
        else:
            return self.calculate_group(id_)
        
        
    def add_new_experiment(self, config_new_exp):
        new_id = max(self.config['experiments'].keys())+1 if self.config else 1
        config_new_exp['params']['base_group'] = []
        
        if config_new_exp['cross_relates']:
            cross_relates = self.config['experiments'][config_new_exp['cross_relates']]['params']['base_group']
            new_p = 1-((len(cross_relates) + 10)/100)
        else:
            cross_relates = []
            new_p = 0.9        
        
        if config_new_exp['p'] > 0.9:
            raise ValueError('Доля не может быть больше 90%, т.к. 10% мы отложили')
        elif config_new_exp['p']>new_p:
            raise ValueError(f'Доля не может быть больше {int(new_p*100)}%, т.к. 10% мы отложили, а также есть пересечения с другими экспериментами')
        
        while len(config_new_exp['params']['base_group']) < int(config_new_exp['p']*100):
            tmp_group = np.random.randint(0,100)
            if tmp_group not in config_new_exp['params']['base_group'] and tmp_group not in cross_relates and tmp_group>9:
                config_new_exp['params']['base_group'].append(tmp_group)
        
        self.config['experiments'][new_id] = config_new_exp

In [394]:
ss = super_service(config, cash_ids, features_cascade)

In [395]:
ss.add_new_experiment({'name': 'Эксперимент 2',
                  'description': 'Второй эксперимент, не связанный с первым',
                  'salt': 61,
                  'p': 0.3,
                  'size': 200000,
                  'actual_size':0,
                  'status':'active',
                  'groups': [{'group1':{'description':'Группа 1 - свойства = Синяя кнопка'}},
                             {'group2':{'description':'Группа 2 - свойства = Красная кнопка'}},
                            ],
                 'cross_relates':None,
                 'params':{}})

In [396]:
ss.add_new_experiment({'name': 'Эксперимент 2',
                  'description': 'Третий эксперимент, пересекающийся с первым',
                  'salt': 40,
                  'p': 0.3,
                  'size': 200000,
                  'actual_size':0,
                  'status':'active',
                  'groups': [{'group1':{'description':'Группа 1 - свойства = Размер страницы 60%'}},
                             {'group2':{'description':'Группа 2 - свойства = Размер страницы 120%'}},
                            ],
                 'cross_relates':1,
                 'params':{}})

In [400]:
for i in device_id:
    ss.get_group(i)

In [401]:
import pandas as pd

In [402]:
df = pd.DataFrame(ss.cash_ids.values())
df['experiments'] = [str(i) for i in df['experiments']]

In [391]:
df.groupby('experiments').count()

,base_group
experiments,
"{1: {'group1'}, 2: {'group1'}, 3: {}}",2953
"{1: {'group1'}, 2: {'group2'}, 3: {}}",992
"{1: {'group1'}, 2: {}, 3: {}}",5963
"{1: {'group2'}, 2: {'group1'}, 3: {}}",2974
"{1: {'group2'}, 2: {'group2'}, 3: {}}",1001
"{1: {'group2'}, 2: {}, 3: {}}",6038
"{1: {'group3'}, 2: {'group1'}, 3: {}}",3042
"{1: {'group3'}, 2: {'group2'}, 3: {}}",1002
"{1: {'group3'}, 2: {}, 3: {}}",6035


In [ ]:
ss.cash_ids

{1: {'base_group': 1, 'experiments': {}},
 770810486: {'base_group': 86, 'experiments': {1: {}, 2: {}, 3: {}}},
 450941023: {'base_group': 23, 'experiments': {1: {}, 2: {}, 3: {'group2'}}},
 345323963: {'base_group': 63, 'experiments': {1: {}, 2: {}, 3: {}}},
 87119338: {'base_group': 38,
  'experiments': {1: {}, 2: {'group2'}, 3: {'group1'}}},
 41464514: {'base_group': 14, 'experiments': {1: {}, 2: {}, 3: {}}},
 913374659: {'base_group': 59, 'experiments': {1: {}, 2: {}, 3: {}}},
 808820665: {'base_group': 65, 'experiments': {1: {}, 2: {}, 3: {'group2'}}},
 29630827: {'base_group': 27, 'experiments': {1: {}, 2: {'group1'}, 3: {}}},
 579741167: {'base_group': 67, 'experiments': {1: {}, 2: {}, 3: {}}},
 193911606: {'base_group': 6, 'experiments': {}},
 224738424: {'base_group': 24, 'experiments': {1: {}, 2: {}, 3: {'group1'}}},
 365523265: {'base_group': 65, 'experiments': {1: {}, 2: {}, 3: {'group2'}}},
 909450149: {'base_group': 49, 'experiments': {1: {}, 2: {}, 3: {}}},
 524752856: {